## Constraint-aware semantic resolution over OMOP vocabularies

* Pydantic classes (via linkml) produce structured LLM queries
* the LLM proposes candidates
* LinkML defines valid target spaces
* the KG enforces ontological correctness
* scoring / ranking resolves ambiguity

### Reasoners

1. Hierarchy reasoner: Strict checks for candidate ⊑ allowed_parent
2. Domain reasoner: Broader checks simply confirming if this concept is in the correct OMOP domain?
3. Vocabulary preferences: List of ordered vocab targets 
4. Standardness and validity
5. Specificity / depth: Out of concepts matching selected restrictions, which is at the most appropriate level of specificity?

In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from dotenv import load_dotenv

from omop_graph.graph.scoring import find_shortest_paths, rank_paths, explain_path
from omop_graph.graph.traverse import traverse
from omop_graph.graph.paths import find_shortest_paths
from omop_graph.graph.kg import KnowledgeGraph
from omop_graph.graph.edges import PredicateKind
from omop_graph.render import (
    render_subgraph,
    render_trace,
    render_path,
    render_explained_path,
    bind_default_renderers,
)

from omop_alchemy import configure_logging, get_engine_name, TEST_PATH, ROOT_PATH
import sqlalchemy as sa
from omop_alchemy.cdm.model.vocabulary import Concept, Concept_Ancestor, Concept_Relationship, Concept_Synonym
import pandas as pd

In [2]:
configure_logging()
load_dotenv()

engine_string = get_engine_name()
engine = sa.create_engine(engine_string, future=True, echo=False)

2026-01-05 22:52:17,245 | INFO     | omop_alchemy.omop_alchemy.config | Database engine configured


In [3]:
Session = sessionmaker(bind=engine)

session = Session()
kg = KnowledgeGraph(session)
bind_default_renderers(kg)

In [4]:
# synonyms = pd.DataFrame(
#     session.query(
#         Concept_Synonym.concept_synonym_name, Concept_Synonym.concept_id, Concept_Synonym.language_concept_id
#         )
# )

In [ ]:
kg.label_lookup("Heart attack")

In [ ]:
sorted(
    kg.label_lookup("Coronary artery thrombosis", fuzzy=False)
)

In [ ]:
sorted(
    kg.label_lookup("Coronary artery thrombosis", fuzzy=True)
)

In [ ]:
from omop_graph.reasoning.term_grounding import GroundingConstraints, ground_term
from omop_graph.reasoning.resolvers import ResolverPipeline, ExactLabelResolver, PartialLabelResolver, ExactSynonymResolver
from omop_graph.reasoning.term_grounding import _passes_constraints


In [ ]:
constraints = GroundingConstraints(
    parent_ids=(4027255,),#(37153816,),         # malignant neoplasm
    allowed_domains=("Condition",), # redundant with parent_ids but perhaps we support combinations across domains eventually for more abstract grounding?
    allowed_vocabularies=("SNOMED", "ICD10CM"),
    require_standard=True,
    max_depth=4,
)

In [ ]:
resolver_pipeline = ResolverPipeline(
    resolvers=(ExactLabelResolver(),ExactSynonymResolver(),PartialLabelResolver()),
    stop_after_confidence=PartialLabelResolver.confidence
)

In [ ]:
ground_term(kg, "heart attack", constraints=constraints, resolver_pipeline=resolver_pipeline)

In [ ]:
e = ExactLabelResolver()
p = PartialLabelResolver()
s = ExactSynonymResolver()

In [ ]:
hits = e.resolve(kg, "heart attack")

In [ ]:
for h in hits:
    ok, reasons = _passes_constraints(kg, h.concept_id, constraints)
    print(f"Concept ID {h.concept_id} passes: {ok}, reasons: {reasons}")

In [ ]:
hits = s.resolve(kg, "heart attack")

In [ ]:
for h in hits:
    ok, reasons = _passes_constraints(kg, h.concept_id, constraints)
    print(f"Concept ID {h.concept_id} passes: {ok}, reasons: {reasons}")